# SUMO Traffic Simulation Setup Tutorial

This notebook provides a comprehensive guide to setting up SUMO (Simulation of Urban Mobility) for traffic optimization using genetic algorithms.

## Overview

SUMO is an open-source traffic simulation package that allows us to:
- Create realistic traffic networks
- Simulate vehicle movements
- Control traffic lights programmatically
- Collect performance metrics

This tutorial will walk you through:
1. Installing SUMO
2. Creating a basic network
3. Setting up traffic flows
4. Connecting to SUMO via TraCI (Python interface)
5. Implementing basic traffic light control

## 1. Installing SUMO

SUMO can be installed on macOS using Homebrew or from source. We'll use the Homebrew method for simplicity.

In [15]:
# Install SUMO via Homebrew (run this in terminal)
# brew install sumo

# Verify installation and check SUMO version
import subprocess
import sys

try:
    result = subprocess.run(['sumo', '--version'], capture_output=True, text=True, timeout=10)
    if result.returncode == 0:
        print("SUMO is installed successfully!")
        print(result.stdout)
    else:
        print("SUMO installation verification failed")
        print(result.stderr)
except FileNotFoundError:
    print("SUMO not found. Please install using: brew install sumo")
except subprocess.TimeoutExpired:
    print("Command timed out")

SUMO is installed successfully!
Eclipse SUMO sumo Version 1.23.1
 Build features: Darwin-24.2.0 arm64 AppleClang 16.0.0.16000026 Release FMI Proj GUI Intl SWIG Eigen GDAL FFmpeg OSG GL2PS
 Copyright (C) 2001-2025 German Aerospace Center (DLR) and others; https://sumo.dlr.de

Eclipse SUMO sumo Version 1.23.1 is part of SUMO.
This program and the accompanying materials
are made available under the terms of the Eclipse Public License v2.0
which accompanies this distribution, and is available at
http://www.eclipse.org/legal/epl-v20.html
This program may also be made available under the following Secondary
Licenses when the conditions for such availability set forth in the Eclipse
Public License 2.0 are satisfied: GNU General Public License, version 2
or later which is available at
https://www.gnu.org/licenses/old-licenses/gpl-2.0-standalone.html
SPDX-License-Identifier: EPL-2.0 OR GPL-2.0-or-later



In [ ]:
# Install required Python packages
import subprocess
import sys

packages = ['deap', 'numpy', 'matplotlib', 'pandas']

for package in packages:
    try:
        __import__(package)
        print(f"✓ {package} is already installed")
    except ImportError:
        print(f"Installing {package}...")
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', package])
        print(f"✓ {package} installed successfully")

# Note: traci comes with SUMO installation

## 2. Creating a Basic SUMO Network

SUMO networks consist of several key files:
- **Network file (.net.xml)**: Defines roads, intersections, and traffic lights
- **Route file (.rou.xml)**: Defines vehicle routes and traffic flows
- **Configuration file (.sumocfg)**: Combines all components for simulation

Let's create a simple 4-way intersection network.

In [10]:
import os

# Create SUMO configuration directory
sumo_dir = "sumo_config"
os.makedirs(sumo_dir, exist_ok=True)

# Create a simple node file (intersections and endpoints)
nodes_content = """<?xml version="1.0" encoding="UTF-8"?>
<nodes xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:noNamespaceSchemaLocation="http://sumo.dlr.de/xsd/nodes_file.xsd">
    <!-- Intersection node -->
    <node id="center" x="0.0" y="0.0" type="traffic_light"/>
    
    <!-- End nodes -->
    <node id="north" x="0.0" y="200.0"/>
    <node id="south" x="0.0" y="-200.0"/>
    <node id="east" x="200.0" y="0.0"/>
    <node id="west" x="-200.0" y="0.0"/>
</nodes>"""

with open(f"{sumo_dir}/intersection.nod.xml", "w") as f:
    f.write(nodes_content)

print("✓ Created nodes file: intersection.nod.xml")

✓ Created nodes file: intersection.nod.xml


In [11]:
# Create edges file (roads connecting nodes)
edges_content = """<?xml version="1.0" encoding="UTF-8"?>
<edges xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:noNamespaceSchemaLocation="http://sumo.dlr.de/xsd/edges_file.xsd">
    <!-- Incoming edges to intersection -->
    <edge id="north_to_center" from="north" to="center" numLanes="2" speed="13.89"/>
    <edge id="south_to_center" from="south" to="center" numLanes="2" speed="13.89"/>
    <edge id="east_to_center" from="east" to="center" numLanes="2" speed="13.89"/>
    <edge id="west_to_center" from="west" to="center" numLanes="2" speed="13.89"/>
    
    <!-- Outgoing edges from intersection -->
    <edge id="center_to_north" from="center" to="north" numLanes="2" speed="13.89"/>
    <edge id="center_to_south" from="center" to="south" numLanes="2" speed="13.89"/>
    <edge id="center_to_east" from="center" to="east" numLanes="2" speed="13.89"/>
    <edge id="center_to_west" from="center" to="west" numLanes="2" speed="13.89"/>
</edges>"""

with open(f"{sumo_dir}/intersection.edg.xml", "w") as f:
    f.write(edges_content)

print("✓ Created edges file: intersection.edg.xml")

✓ Created edges file: intersection.edg.xml


In [12]:
# Generate network file using netconvert
import subprocess

try:
    cmd = [
        'netconvert',
        '--node-files', f'{sumo_dir}/intersection.nod.xml',
        '--edge-files', f'{sumo_dir}/intersection.edg.xml',
        '--output-file', f'{sumo_dir}/intersection.net.xml',
        '--tls.guess', 'true'  # Automatically generate traffic light logic
    ]
    
    result = subprocess.run(cmd, capture_output=True, text=True)
    
    if result.returncode == 0:
        print("✓ Network file created successfully: intersection.net.xml")
        if result.stdout:
            print("Output:", result.stdout)
    else:
        print("❌ Error creating network file:")
        print(result.stderr)
        
except FileNotFoundError:
    print("❌ netconvert not found. Make sure SUMO is installed and in PATH")
except Exception as e:
    print(f"❌ Error: {e}")

❌ Error creating network file:
dyld[73459]: Library not loaded: /opt/homebrew/opt/gdal/lib/libgdal.36.dylib
  Referenced from: <630D8B34-9361-3654-A176-4CB7C338C1BE> /opt/miniconda3/lib/python3.12/site-packages/sumo/bin/netconvert
  Reason: tried: '/opt/homebrew/opt/gdal/lib/libgdal.36.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/opt/gdal/lib/libgdal.36.dylib' (no such file), '/opt/homebrew/opt/gdal/lib/libgdal.36.dylib' (no such file), '/opt/homebrew/Cellar/gdal/3.11.1_1/lib/libgdal.36.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/Cellar/gdal/3.11.1_1/lib/libgdal.36.dylib' (no such file), '/opt/homebrew/Cellar/gdal/3.11.1_1/lib/libgdal.36.dylib' (no such file)



## 3. Creating Vehicle Routes and Traffic Flows

Now we'll create routes for vehicles and define traffic flows to simulate realistic traffic patterns.

In [13]:
# Create route file with different vehicle flows
routes_content = """<?xml version="1.0" encoding="UTF-8"?>
<routes xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:noNamespaceSchemaLocation="http://sumo.dlr.de/xsd/routes_file.xsd">
    
    <!-- Vehicle type definition -->
    <vType id="car" accel="2.6" decel="4.5" sigma="0.5" length="5" maxSpeed="50"/>
    
    <!-- Route definitions (straight movements) -->
    <route id="north_south" edges="north_to_center center_to_south"/>
    <route id="south_north" edges="south_to_center center_to_north"/>
    <route id="east_west" edges="east_to_center center_to_west"/>
    <route id="west_east" edges="west_to_center center_to_east"/>
    
    <!-- Route definitions (turning movements) -->
    <route id="north_east" edges="north_to_center center_to_east"/>
    <route id="north_west" edges="north_to_center center_to_west"/>
    <route id="south_east" edges="south_to_center center_to_east"/>
    <route id="south_west" edges="south_to_center center_to_west"/>
    <route id="east_north" edges="east_to_center center_to_north"/>
    <route id="east_south" edges="east_to_center center_to_south"/>
    <route id="west_north" edges="west_to_center center_to_north"/>
    <route id="west_south" edges="west_to_center center_to_south"/>
    
    <!-- Traffic flows (vehicles per hour) -->
    <flow id="flow_ns" route="north_south" begin="0" end="3600" vehsPerHour="400" type="car"/>
    <flow id="flow_sn" route="south_north" begin="0" end="3600" vehsPerHour="400" type="car"/>
    <flow id="flow_ew" route="east_west" begin="0" end="3600" vehsPerHour="300" type="car"/>
    <flow id="flow_we" route="west_east" begin="0" end="3600" vehsPerHour="300" type="car"/>
    
    <!-- Turning movements (lower frequency) -->
    <flow id="flow_ne" route="north_east" begin="0" end="3600" vehsPerHour="100" type="car"/>
    <flow id="flow_nw" route="north_west" begin="0" end="3600" vehsPerHour="100" type="car"/>
    <flow id="flow_se" route="south_east" begin="0" end="3600" vehsPerHour="100" type="car"/>
    <flow id="flow_sw" route="south_west" begin="0" end="3600" vehsPerHour="100" type="car"/>
    <flow id="flow_en" route="east_north" begin="0" end="3600" vehsPerHour="80" type="car"/>
    <flow id="flow_es" route="east_south" begin="0" end="3600" vehsPerHour="80" type="car"/>
    <flow id="flow_wn" route="west_north" begin="0" end="3600" vehsPerHour="80" type="car"/>
    <flow id="flow_ws" route="west_south" begin="0" end="3600" vehsPerHour="80" type="car"/>
    
</routes>"""

with open(f"{sumo_dir}/intersection.rou.xml", "w") as f:
    f.write(routes_content)

print("✓ Created routes file: intersection.rou.xml")
print("Traffic flows defined:")
print("- North-South: 400 veh/h each direction")
print("- East-West: 300 veh/h each direction") 
print("- Turning movements: 80-100 veh/h each")

✓ Created routes file: intersection.rou.xml
Traffic flows defined:
- North-South: 400 veh/h each direction
- East-West: 300 veh/h each direction
- Turning movements: 80-100 veh/h each


In [ ]:
# Create SUMO configuration file
config_content = """<?xml version="1.0" encoding="UTF-8"?>
<configuration xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:noNamespaceSchemaLocation="http://sumo.dlr.de/xsd/sumoConfiguration.xsd">
    
    <input>
        <net-file value="intersection.net.xml"/>
        <route-files value="intersection.rou.xml"/>
    </input>
    
    <time>
        <begin value="0"/>
        <end value="3600"/>
        <step-length value="1"/>
    </time>
    
    <processing>
        <time-to-teleport value="300"/>
    </processing>
    
    <report>
        <verbose value="true"/>
        <no-step-log value="true"/>
    </report>
    
    <traci_server>
        <remote-port value="8813"/>
    </traci_server>
    
</configuration>"""

with open(f"{sumo_dir}/intersection.sumocfg", "w") as f:
    f.write(config_content)

print("✓ Created SUMO configuration file: intersection.sumocfg")
print("Configuration includes:")
print("- Network and route files")
print("- Simulation time: 1 hour")
print("- TraCI server enabled on port 8813")

## 4. Connecting to SUMO via TraCI

TraCI (Traffic Control Interface) allows us to control SUMO simulations programmatically from Python. This is essential for implementing genetic algorithm optimization.

In [14]:
# Test TraCI connection and run basic simulation
import sys
import os

# Add SUMO tools to Python path
try:
    # Try to find SUMO installation
    if 'SUMO_HOME' in os.environ:
        tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
        sys.path.append(tools)
    else:
        # Common SUMO installation paths on macOS
        possible_paths = [
            '/opt/homebrew/share/sumo/tools',
            '/usr/local/share/sumo/tools',
            '/usr/share/sumo/tools'
        ]
        for path in possible_paths:
            if os.path.exists(path):
                sys.path.append(path)
                os.environ['SUMO_HOME'] = path.replace('/tools', '')
                break
    
    import traci
    print("✓ TraCI imported successfully")
    
except ImportError as e:
    print(f"❌ Error importing TraCI: {e}")
    print("Make sure SUMO is installed and SUMO_HOME is set")
    print("You can set it with: export SUMO_HOME=/opt/homebrew/share/sumo")

❌ Error importing TraCI: No module named 'traci'
Make sure SUMO is installed and SUMO_HOME is set
You can set it with: export SUMO_HOME=/opt/homebrew/share/sumo


In [ ]:
# Run a basic SUMO simulation with TraCI
def run_basic_simulation(duration=60):
    """Run a basic SUMO simulation for the specified duration"""
    try:
        # Start SUMO with TraCI
        sumo_binary = "sumo"  # Use "sumo-gui" for graphical version
        sumo_cmd = [sumo_binary, "-c", f"{sumo_dir}/intersection.sumocfg", "--start"]
        
        traci.start(sumo_cmd)
        
        # Get traffic light IDs
        tl_ids = traci.trafficlight.getIDList()
        print(f"Traffic lights found: {tl_ids}")
        
        # Simulation loop
        step = 0
        vehicle_count = 0
        
        print(f"Running simulation for {duration} seconds...")
        
        while step < duration:
            traci.simulationStep()
            
            # Get simulation statistics every 10 steps
            if step % 10 == 0:
                current_vehicles = traci.vehicle.getIDCount()
                if step % 30 == 0:  # Print every 30 seconds
                    print(f"Step {step}: {current_vehicles} vehicles in simulation")
                
                vehicle_count = max(vehicle_count, current_vehicles)
            
            step += 1
        
        print(f"Simulation completed!")
        print(f"Maximum vehicles in simulation: {vehicle_count}")
        
        # Close TraCI connection
        traci.close()
        
    except Exception as e:
        print(f"❌ Error during simulation: {e}")
        try:
            traci.close()
        except:
            pass

# Run the simulation
run_basic_simulation(60)

## 5. Traffic Light Control and Optimization

Now we'll implement basic traffic light control and show how to modify signal timings - this is the foundation for genetic algorithm optimization.

In [ ]:
# Traffic light control and performance measurement
def evaluate_signal_timing(ns_green, ew_green, yellow_time=3, simulation_time=300):
    """
    Evaluate traffic signal timing performance
    
    Args:
        ns_green: Green time for North-South direction (seconds)
        ew_green: Green time for East-West direction (seconds)
        yellow_time: Yellow time for transitions (seconds)
        simulation_time: Total simulation time (seconds)
    
    Returns:
        Performance metrics (average travel time, waiting time, etc.)
    """
    try:
        # Start SUMO
        sumo_cmd = ["sumo", "-c", f"{sumo_dir}/intersection.sumocfg", "--start", "--no-warnings"]
        traci.start(sumo_cmd)
        
        # Get traffic light ID
        tl_ids = traci.trafficlight.getIDList()
        if not tl_ids:
            print("No traffic lights found")
            return None
            
        tl_id = tl_ids[0]
        
        # Create signal program
        # Traffic light phases: N/S green, yellow, E/W green, yellow
        phases = [
            traci.trafficlight.Phase(ns_green, "GGrrGGrr"),  # N/S green
            traci.trafficlight.Phase(yellow_time, "yyrryyrr"),  # N/S yellow
            traci.trafficlight.Phase(ew_green, "rrGGrrGG"),  # E/W green  
            traci.trafficlight.Phase(yellow_time, "rryyrryy")   # E/W yellow
        ]
        
        # Set the new signal program
        program_id = "custom"
        traci.trafficlight.setCompleteRedYellowGreenDefinition(tl_id, 
            traci.trafficlight.Logic(program_id, 0, 0, phases))
        traci.trafficlight.setProgram(tl_id, program_id)
        
        # Simulation metrics
        total_travel_time = 0
        total_waiting_time = 0
        vehicle_count = 0
        departed_vehicles = set()
        
        # Run simulation
        for step in range(simulation_time):
            traci.simulationStep()
            
            # Collect metrics for all vehicles
            for veh_id in traci.vehicle.getIDList():
                if veh_id not in departed_vehicles:
                    departed_vehicles.add(veh_id)
                
                # Get vehicle metrics
                travel_time = traci.vehicle.getDrivingDistance(veh_id)
                waiting_time = traci.vehicle.getWaitingTime(veh_id)
                
                total_travel_time += travel_time
                total_waiting_time += waiting_time
                vehicle_count += 1
        
        # Calculate average metrics
        if vehicle_count > 0:
            avg_travel_time = total_travel_time / vehicle_count
            avg_waiting_time = total_waiting_time / vehicle_count
        else:
            avg_travel_time = float('inf')
            avg_waiting_time = float('inf')
        
        metrics = {
            'avg_travel_time': avg_travel_time,
            'avg_waiting_time': avg_waiting_time,
            'total_vehicles': len(departed_vehicles),
            'cycle_time': ns_green + ew_green + 2 * yellow_time
        }
        
        traci.close()
        return metrics
        
    except Exception as e:
        print(f"Error in simulation: {e}")
        try:
            traci.close()
        except:
            pass
        return None

# Test different signal timings
print("Testing different signal timing configurations...")
print("(This may take a moment...)")

configs = [
    (30, 30),  # Equal timing
    (40, 20),  # Favor N/S
    (20, 40),  # Favor E/W
    (25, 35),  # Slight E/W preference
]

results = []
for ns_time, ew_time in configs:
    print(f"Testing N/S:{ns_time}s, E/W:{ew_time}s...")
    metrics = evaluate_signal_timing(ns_time, ew_time, simulation_time=180)
    if metrics:
        results.append({
            'ns_green': ns_time,
            'ew_green': ew_time,
            'cycle_time': metrics['cycle_time'],
            'avg_waiting_time': metrics['avg_waiting_time'],
            'total_vehicles': metrics['total_vehicles']
        })
        print(f"  → Avg waiting time: {metrics['avg_waiting_time']:.2f}s")

print("\\nSignal timing comparison:")
for result in results:
    print(f"N/S:{result['ns_green']}s, E/W:{result['ew_green']}s → "
          f"Waiting: {result['avg_waiting_time']:.2f}s, "
          f"Vehicles: {result['total_vehicles']}")

## 6. Genetic Algorithm Integration

Now we'll create a simple genetic algorithm to optimize traffic signal timings using the SUMO simulation as the fitness evaluation function.

In [ ]:
from deap import base, creator, tools, algorithms
import random
import numpy as np

# Define the genetic algorithm for signal timing optimization
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))  # Minimize waiting time
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()

# Signal timing constraints
MIN_GREEN = 10  # Minimum green time
MAX_GREEN = 60  # Maximum green time

# Gene = [ns_green_time, ew_green_time]
toolbox.register("attr_green", random.randint, MIN_GREEN, MAX_GREEN)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_green, n=2)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def evaluate_individual(individual):
    """
    Fitness function: evaluate signal timing using SUMO simulation
    Returns average waiting time (to be minimized)
    """
    ns_green, ew_green = individual
    
    # Ensure valid timing constraints
    if ns_green < MIN_GREEN or ns_green > MAX_GREEN:
        return (float('inf'),)
    if ew_green < MIN_GREEN or ew_green > MAX_GREEN:
        return (float('inf'),)
    
    # Run simulation with these timings
    metrics = evaluate_signal_timing(ns_green, ew_green, simulation_time=120)
    
    if metrics is None:
        return (float('inf'),)
    
    # Return waiting time as fitness (to be minimized)
    return (metrics['avg_waiting_time'],)

def mutate_individual(individual, indpb=0.2):
    """Custom mutation: adjust timing by small amounts"""
    for i in range(len(individual)):
        if random.random() < indpb:
            # Mutate by +/- 1 to 5 seconds
            change = random.randint(-5, 5)
            individual[i] = max(MIN_GREEN, min(MAX_GREEN, individual[i] + change))
    return individual,

# Register genetic operators
toolbox.register("evaluate", evaluate_individual)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", mutate_individual)
toolbox.register("select", tools.selTournament, tournsize=3)

print("✓ Genetic Algorithm setup complete")
print(f"Signal timing constraints: {MIN_GREEN}-{MAX_GREEN} seconds")
print("Fitness function: Minimize average waiting time")

In [ ]:
# Run genetic algorithm optimization
def run_traffic_optimization(pop_size=8, generations=5):
    """
    Run genetic algorithm to optimize traffic signal timings
    Note: Using small population and generations for demonstration
    """
    print(f"Starting traffic signal optimization...")
    print(f"Population size: {pop_size}, Generations: {generations}")
    print("This will take several minutes as each individual requires a SUMO simulation...")
    
    # Create initial population
    pop = toolbox.population(n=pop_size)
    
    # Statistics tracking
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("min", np.min)
    stats.register("avg", np.mean)
    
    # Hall of fame to keep best individual
    hof = tools.HallOfFame(1)
    
    # Evolution parameters
    CXPB = 0.7  # Crossover probability
    MUTPB = 0.3  # Mutation probability
    
    # Evaluate initial population
    print("\\nEvaluating initial population...")
    fitnesses = []
    for i, ind in enumerate(pop):
        print(f"  Individual {i+1}/{len(pop)}: N/S={ind[0]}s, E/W={ind[1]}s", end="")
        fitness = toolbox.evaluate(ind)
        ind.fitness.values = fitness
        fitnesses.append(fitness[0])
        print(f" → Waiting time: {fitness[0]:.2f}s")
    
    hof.update(pop)
    
    # Evolution loop
    for gen in range(generations):
        print(f"\\n=== Generation {gen + 1} ===")
        
        # Selection and breeding
        offspring = toolbox.select(pop, len(pop))
        offspring = list(map(toolbox.clone, offspring))
        
        # Crossover
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values
        
        # Mutation
        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values
        
        # Evaluate invalid individuals
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        for i, ind in enumerate(invalid_ind):
            print(f"  Evaluating {i+1}/{len(invalid_ind)}: N/S={ind[0]}s, E/W={ind[1]}s", end="")
            fitness = toolbox.evaluate(ind)
            ind.fitness.values = fitness
            print(f" → Waiting time: {fitness[0]:.2f}s")
        
        # Replace population
        pop[:] = offspring
        hof.update(pop)
        
        # Statistics
        fits = [ind.fitness.values[0] for ind in pop]
        print(f"  Best: {min(fits):.2f}s, Average: {np.mean(fits):.2f}s")
    
    # Results
    best_individual = hof[0]
    print(f"\\n🎯 OPTIMIZATION COMPLETE!")
    print(f"Best signal timing found:")
    print(f"  North-South green: {best_individual[0]} seconds")
    print(f"  East-West green: {best_individual[1]} seconds")
    print(f"  Average waiting time: {best_individual.fitness.values[0]:.2f} seconds")
    
    return best_individual

# Run optimization (small scale for demonstration)
print("⚠️  Note: This optimization uses a small population and few generations for demonstration.")
print("For real optimization, use larger values (pop_size=50, generations=20+)")
print("\\nStarting optimization...")

best_timing = run_traffic_optimization(pop_size=6, generations=3)

## 7. Summary and Next Steps

### What We've Accomplished

✅ **SUMO Installation and Setup**
- Installed SUMO traffic simulation software
- Created a basic 4-way intersection network
- Defined vehicle routes and traffic flows

✅ **TraCI Integration**  
- Connected Python to SUMO via TraCI interface
- Implemented simulation control and data collection
- Created traffic light control functions

✅ **Genetic Algorithm Implementation**
- Built GA using DEAP library
- Defined fitness function based on waiting time
- Demonstrated signal timing optimization

### Performance Insights

The optimization process shows how genetic algorithms can:
- Automatically discover good signal timing combinations
- Balance competing traffic flows (North-South vs East-West)
- Minimize overall waiting times through evolution

### Next Steps for Full Implementation

**1. Enhanced Network Modeling**
- Multi-intersection networks
- Realistic traffic demand patterns
- Variable vehicle types and behaviors

**2. Improved Optimization**
- Larger population sizes (50-100 individuals)
- More generations (20-50)
- Multi-objective optimization (waiting time + throughput + emissions)

**3. Advanced Features**
- Real-time traffic adaptation
- Pedestrian crossing integration
- Emergency vehicle priority

**4. Validation and Analysis**
- Compare with existing signal timing methods
- Statistical significance testing
- Visualization of results

### File Structure Created

```
sumo_config/
├── intersection.nod.xml     # Network nodes
├── intersection.edg.xml     # Network edges  
├── intersection.net.xml     # Complete network
├── intersection.rou.xml     # Vehicle routes
└── intersection.sumocfg     # SUMO configuration
```

This tutorial provides the foundation for implementing sophisticated traffic optimization using genetic algorithms and SUMO simulation!